In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import firedrake

### Meshes

First, we'll create a mesh of the unit square.
Later, I'll show an example where we use a more interesting geometry.

In [ ]:
mesh = firedrake.UnitSquareMesh(20, 20)

Here I'll make a plot of the mesh we just created.
It's very common to need different boundary conditions on different parts of the domain.
To make that easy, you can assign numeric identifies to each part of the boundary.
The plot below uses different colors for each boundary segment with a legend so you can tell which is which.

In [ ]:
fig, axes = plt.subplots()
axes.set_aspect('equal')
firedrake.triplot(mesh, axes=axes)
axes.legend()

### Functions and function spaces

Now let's create a function space defined on that mesh.
Here we'll use continuous Galerkin (`'CG'`) finite elements.
There are also various families of discontinuous elements and we'll see a bit of that later.
We'll pick the polynomial degree to be 2, but you could just as easily use 1 or 3 or whatever you like.

In [ ]:
Q = firedrake.FunctionSpace(mesh, family='CG', degree=2)

Now we'll create the right-hand side for our problem, which in this case will be the function

$$f(x, y) = x^2 - y^2.$$

To do this, we'll first use the function `SpatialCoordinate` on the mesh to get two variables `x`, `y` which represent the position of a point within the mesh.
These variables are symbolic objects which we can manipulate algebraically to form more interesting expressions.
We'll use that capability to create a variable `f` that represents our right-hand side.

In [ ]:
x, y = firedrake.SpatialCoordinate(mesh)
f = x**2 - y**2

The variable `f` is again a *symbolic* object -- it doesn't have an expansion coefficients, there's no big array of number living underneath it.
We can convert it into a numeric object by *interpolating* it into the finite element space $Q$ that we created above.

In [ ]:
F = firedrake.interpolate(f, Q)

The variable `F` does have a big array of expansion coefficients under the hood:

In [ ]:
F.dat.data_ro

We can also plot it:

In [ ]:
fig = plt.figure()
axes = fig.add_subplot(projection='3d')
firedrake.trisurf(F, axes=axes)

### Evaluating integrals

Let's do one more thing before we actually get down to solving PDEs: evaluating integrals.
To evaluate the integral of a function on some domain, we multiply that function by the symbol `dx`.
Much like the `x` and `y` variables we created before, `dx` is a symbolic object that tells Firedrake that we're about to integrate something.

In [ ]:
from firedrake import dx
integral = f * dx

Once again, the object `f * dx` is just symbolic -- it represents something that we can integrate, it isn't a floating point number yet.
To get a number, we use the function `assemble`:

In [ ]:
from firedrake import assemble
assemble(integral)

We can do algebraic manipulations on `f` as well:

In [ ]:
assemble(f**2 * dx)

Instead of integrating over the whole domain, we could also integrate over just the boundary by using the symbol `ds` instead of `dx`:

In [ ]:
from firedrake import ds
assemble(f**2 * ds)

To use the weighted residual method, you need to be able to evaluate integrals, so you can see why this is a pretty important functionality to have.

### Solving PDEs

Before, we created symbolic objects `x` and `y` to represent points within our domain, and we created expressions by manipulating them algebraically.
We then used these expressions to create functions.
To define what PDE we want to solve, we'll proceed in a similar way, but instead we'll be creating symbolic objects to represent the trial functions (the solution) and the test functions (the things we want our solution to be orthogonal to).

In [ ]:
u, v = firedrake.TrialFunction(Q), firedrake.TestFunction(Q)

When we created expressions out of the spatial coordinates, we could use all the usual algebraic operations (adding, multiplying, exponentiating) and some transcendental functions too (sin, cos, exp).
When we create weak forms out of test and trial functions, we'll want to talk about their derivatives.
To do that we use the function `grad` or gradient.
The gradient of a scalar field is a vector field; the function `inner` calculates the inner product of two vectors or vector fields.

In [ ]:
from firedrake import inner, grad
a = inner(grad(u), grad(v)) * dx
L = f * v * dx

To have a well-posed problem, we'll also need some boundary conditions.
Here we'll specify that the solution is fixed to zero around all edges of the domain, but you can play with this later.

In [ ]:
bc = firedrake.DirichletBC(Q, 0, 'on_boundary')

Now the trial and test functions are symbolic objects, but if we want to actually solve the problem we need a real honest numeric function that has an array of expansion coefficients somewhere under the hood.
So we'll create a new variable `u` that lives in the function space `Q`; this is where the solution will go.
Finally, we'll pass our problem, the function where we want to store the solution, and the boundary conditions to `firedrake.solve`.

In [ ]:
u = firedrake.Function(Q)
firedrake.solve(a == L, u, bc)

And a visualization of the result.

In [ ]:
fig = plt.figure()
axes = fig.add_subplot(projection='3d')
firedrake.trisurf(u, axes=axes)